In [1]:
import jsonlines as jl
import nltk
import xmltodict
from ispras import texterra as isp
import pymorphy2 as pm2
import gensim
import time as tm
from lxml import etree
import sys

isp_api = isp.API('ba74236a7212a71054ae1408b30b1bdef771d35b')
#nltk.download()

tokenizer = nltk.TweetTokenizer()
morph = pm2.MorphAnalyzer()  

C:\Users\Maxim\Anaconda3\lib\site-packages\gensim-2.2.0-py3.5-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Лемматизация с помощью Texterra

Читаю комментарии и статьи из файла по очереди, разбиваю их на предложения, лемматизирую с помощью Texterra, составляя при этом словарь FullVocab. Записи в Fullvocab хранятся в следующем виде:

__item__:
- __id__: id нормальной формы
- __normal_form__: нормальная форма
- __tags__: часть речи
- __word_forms__: список встретившихся словоформ
- __cnt__: сколько раз встречалось
- __origin__: в каком корпусе текстов слово встретилось (t - треннировочный [комментарии], a - целевой [статьи], e - дополнительный [например, вешний словарь])

В lemmatized.txt записываются построчно лемматизированные предложения, но вместо слов в нём "id нормальной формы", чтобы для word2vec слова "стать" (<i>глагол</i>) и "стать" (<i>сущ</i>) не были одним и тем же словом.

In [2]:
class FullVocab:
    
    # Инициализация, либо пустой класс, либо из файла vocab.xml
    def __init__(self, from_file = False):
        self.id_counter = 0
        self.vocab = []
        if from_file:
            f = open('vocab.xml', 'rb')
            root = etree.XML(f.read())
            for item in root:
                self.add_item(item[3].text.split(' '), item[1].text, item[2].text, item[5].text, cnt = item[4].text, 
                              word_id = item[0].text)
            
        
    # Добавляем новое слово в словарь
    def add_item(self, words, normal_form, tags_normal, origin, cnt = 0, word_id = -1):
        
        # Дабы было можно и str и list в аргументы добавлять
        if type(words) is not list:
            words = [words]
            
        # Проверяем, есть ли нормальная форма слова в словаре. Если есть, то обновляем словоформы, если нет, то 
        # добавляем новую запись в словарь, присвивая уникальный id
        for item in self.vocab:
            if item.get('normal_form') == normal_form and item.get('tags') == tags_normal:
                item['word_forms'].extend(words)
                item['word_forms'] = sorted(set(item['word_forms']))
                item['cnt'] += 1
                item['origin'] = self.handle_origin(origin, item['origin'])
                return item['id']
                
        #Если такой записи ещё не было
        new_item = {'normal_form': normal_form, 'tags': tags_normal, 'word_forms': words}
        if normal_form not in new_item['word_forms']:
            new_item['word_forms'].append(normal_form)
        if int(cnt) > 0:
            new_item['cnt'] = int(cnt)
        else:
            new_item['cnt'] = 1
            
        new_item['origin'] = self.handle_origin(origin)
        
        return_id = 0
        if word_id == -1:         #если мы не уточняем, какой именно id у нового слова (когда просто добавляем новое слово)
            new_item['id'] = self.id_counter
            return_id = self.id_counter
            self.id_counter = self.id_counter + 1
        else:                     #если мы указали id для нового слова (когда читаем из файла сформированный словарь)
            return_id = int(word_id)
            new_item['id'] = return_id
            self.id_counter = return_id + 1
            
        self.vocab.append(new_item)
        return return_id
         
        
    def show_vocab(self):
        print(self.vocab)
        
        
    # Сохраняем копию данные класса в vocab.xml
    def build_xml(self):
        f = open('vocab.xml', 'wb')
        root = etree.Element('root')
        for item in self.vocab:
            child = etree.SubElement(root, 'w' + str(item['id']))
            etree.SubElement(child, 'id').text = str(item['id'])
            etree.SubElement(child, 'normal_form').text = item['normal_form']
            etree.SubElement(child, 'tags').text = item['tags']
            etree.SubElement(child, 'word_forms').text = ' '.join(item['word_forms'])
            etree.SubElement(child, 'cnt').text = str(item['cnt'])
            etree.SubElement(child, 'origin').text = item['origin']
            
        result = etree.tostring(root, encoding='unicode', method='xml', pretty_print=True)
        f.write(result.encode('utf8'))
        
        
    # Ищем нормальную форму или запись в словаре по id
    # Если return_val = "word", то возвращаем нормальную форму, если "item", то запись в словаре
    def word_by_id(self, word_id, return_val = "word"):
        item = next((item for item in self.vocab if item["id"] == word_id), None)
        if item is None:
            return None
        elif return_val == 'word':
            return item['normal_form']
        elif return_val == 'item':
            return item
        
        
    def id_by_word(self, word, tag = None):
        if tag == None:
            item = next((item for item in self.vocab if item['normal_form'] == word), None)
        else:
            item = next((item for item in self.vocab if item['normal_form'] == word and item['tags'] == tag), None)
        if item is None:
            return None
        else:
            return item['id']
    
    
    def handle_origin(self, new, old = 'xxx'):
        if len(new) == 1:
            if new == 't':
                return 't' + old[1:3]
            elif new == 'a':
                return old[0] + 'a' + old[2]
            elif new == 'e':
                return  old[0:2] + 'e'
            
        elif len(new) == 3:
            return new
        
    
    #Почему-то не работает, использовать cell, который будет ниже
    def print_slang_vocab(self, model):
        synonyms_cnt = 30
        with jl.open('slang_vocab.txt', 'w') as fd:
            for item in self.vocab:
                if item['origin'] in ['tax', 'xax'] and item['cnt'] > 10:
                    synonyms = model.most_similar_cosmul(positive = [str(item['id'])], topn = synonyms_cnt)
                    suitable_syns = [s for s in synonyms if self.word_by_id(int(s[0]), return_val = 'item')['origin'] in
                                   ['txx'] and 
                                   self.word_by_id(int(s[0]), return_val = 'item')['tags'] == item['tags'] and 
                                   not (self.word_by_id(int(s[0])) == 'не' + item['normal_form'] or
                                   'не' + self.word_by_id(int(s[0])) == item['normal_form'] or
                                   self.word_by_id(int(s[0])) == 'без' + item['normal_form'] or
                                   'без' + self.word_by_id(int(s[0])) == item['normal_form'])]
                    if(len(suitable_syns) > 0):
                        line = item['normal_form'] + ' - '
                        syns = [self.word_by_id(int(s[0])) for s in suitable_syns]
                        line += ', '.join(syns)
                        #print(line)
                        fd.write(line)

In [3]:
#Раскоментить, чтобы составить словарь и файл с лемматизированным текстом из скаченных scrapy файлов
'''
lemmatized_txt = open('lemmatized.txt', 'w', encoding='utf8')

with open('learn_scrapy\pda.jl', 'rb') as f:
    
    vocab = FullVocab()
    num_comments = 100617
    counter = 0
    prev_counter = 0
    anchor_counter = 0
    checkpoint = tm.clock()
    dt = 900
    
    for item in jl.reader(f):
        item_origin = None    #Сюда пишем 't', если comment, 'a', если article
        sents = None
        
        if item.get('comment') is not None:
            sents = nltk.sent_tokenize(item['comment'])
            item_origin = 't'
        elif item.get('article') is not None:   
            sents = nltk.sent_tokenize(item['article'])
            item_origin = 'a'
            
        for sent in sents:
            toks = tokenizer.tokenize(sent)
            toks = [t.lower() for t in toks if t.isalnum()]
            sent = ' '.join(toks)
            
            try:
                lemma_annotate = isp_api.lemmatizationAnnotate(text = sent)
                pos_annotate = isp_api.posTaggingAnnotate(text = sent)
                
            except KeyboardInterrupt:
                print("Understandable, have a nice day")
                vocab.build_xml()
                lemmatized_txt.close()
                sys.exit()
                
            except:
                print("Sentence", end = " \"")
                print(sent, end = "\" ")
                print("in commentary #", end = "")
                print(counter, end = " ")
                print("not processed")
                print("\n")
                continue
                
            if lemma_annotate.get('annotations').get('lemma') is not None:
                lemmatized = []
                for i in range(len(lemma_annotate.get('annotations').get('lemma'))):
                    l = lemma_annotate.get('annotations').get('lemma')[i].get('value')
                    p = pos_annotate['annotations']['pos-token'][i]['value'].get('tag')
                    word_id = vocab.add_item(toks[i], l, p, item_origin)
                    lemmatized.append(str(word_id))
                
                lemmatized = ' '.join(lemmatized)
                lemmatized_txt.write(lemmatized + '\n')
        
        counter = counter + 1
        if(tm.clock() - checkpoint > dt):
            print("Progress: ", end = '')
            print(counter / num_comments * 100, end = '')
            print("%")
            print("Comments processed by turn: ", end= '')
            print(counter - prev_counter)
            print("Comments processed total: ", end= '')
            print(counter)
            print("Time left: ", end = '')
            speed = (counter - prev_counter) / dt
            print(speed * (num_comments - counter))
            print("Speed: ", end = '')
            print(speed, end = ' ')
            print("comments/sec")
            print("\n")
            prev_counter = counter
            vocab.build_xml()
            if (counter - anchor_counter) / num_comments * 100 > 25:
                print("Sleeping for 20 minutes")
                tm.sleep(1200)
                anchor_counter = counter
            checkpoint = tm.clock()
            
    vocab.build_xml()
    print("Finished")
    
lemmatized_txt.close()
'''

'\nlemmatized_txt = open(\'lemmatized.txt\', \'w\', encoding=\'utf8\')\n\nwith open(\'learn_scrapy\\pda.jl\', \'rb\') as f:\n    \n    vocab = FullVocab()\n    num_comments = 100617\n    counter = 0\n    prev_counter = 0\n    anchor_counter = 0\n    checkpoint = tm.clock()\n    dt = 900\n    \n    for item in jl.reader(f):\n        item_origin = None    #Сюда пишем \'t\', если comment, \'a\', если article\n        sents = None\n        \n        if item.get(\'comment\') is not None:\n            sents = nltk.sent_tokenize(item[\'comment\'])\n            item_origin = \'t\'\n        elif item.get(\'article\') is not None:   \n            sents = nltk.sent_tokenize(item[\'article\'])\n            item_origin = \'a\'\n            \n        for sent in sents:\n            toks = tokenizer.tokenize(sent)\n            toks = [t.lower() for t in toks if t.isalnum()]\n            sent = \' \'.join(toks)\n            \n            try:\n                lemma_annotate = isp_api.lemmatizationAnno

# Word2vec модель из gensim
Ищу синонимы с помощью метода most_similar() и подставляю в изначальный текст

In [4]:
#Загружаю словарь из файла
new_vocab = FullVocab(from_file = True)
print('Vocab finished')

Vocab finished


In [6]:
#Строю модель из лемматизированных комментариев и/или статей
w2v_model = 0
w2v_model = gensim.models.Word2Vec(gensim.models.word2vec.LineSentence('lemmatized.txt'), min_count=10)
print('Building models completed')

Building models completed


In [ ]:
#Запустить, чтобы записать в файлик slang_vocab.txt сленговый словарь в виде "слово из статей" - ["список сленговых синонимов"]
synonyms_cnt = 30
with jl.open('slang_vocab.txt', 'w') as fd:
    for item in new_vocab.vocab:
        if item['origin'] in ['tax', 'xax'] and item['cnt'] > 10 and item['normal_form'] is not None:
            synonyms = w2v_model.most_similar_cosmul(positive = [str(item['id'])], topn = synonyms_cnt)
            suitable_syns = [s for s in synonyms if new_vocab.word_by_id(int(s[0]), return_val = 'item')['origin'] in
                           ['txx'] and 
                           new_vocab.word_by_id(int(s[0]), return_val = 'item')['tags'] == item['tags'] and 
                           not (new_vocab.word_by_id(int(s[0])) == 'не' + item['normal_form'] or
                           'не' + new_vocab.word_by_id(int(s[0])) == item['normal_form'] or
                           new_vocab.word_by_id(int(s[0])) == 'без' + item['normal_form'] or
                           'без' + new_vocab.word_by_id(int(s[0])) == item['normal_form'])]
            if(len(suitable_syns) > 0):
                line = item['normal_form'] + ' - '
                syns = [new_vocab.word_by_id(int(s[0])) for s in suitable_syns]
                line += ', '.join(syns)
                #print(line)
                fd.write(line)

#new_vocab.print_slang_vocab(w2v_model)
print("Finished building slang vocabulary")

In [50]:
#Получаю наилучший вариант из pymorphy2.parse
def get_propper_parse(word, word_forms):
    word_parse = morph.parse(word)
    ranks = [0] * len(word_parse)
    for i, p in enumerate(word_parse):
        lexemas = [w.word for w in p.lexeme]
        for w in word_forms:
            if w in lexemas:
                ranks[i] += 1

    max_index = ranks.index(max(ranks))
    return word_parse[max_index]


#Спрягаю word по подобию template
def inflect_by_template(word, template):
    grammemes = sorted(template.tag.grammemes)
    for g in grammemes:
        w = word.inflect({g})
        if w is not None:
            word = w

    return word


#Принимаем решение, пропускаем ли синоним
def make_decision(suitable_syns, word_item, cnt):
    decision = False
    freq = int(word_item['cnt']) / cnt
    first_proximity = 0.79
    second_proximity = 0.75
    allowed_freq_min = 100 / cnt
    allowed_freq_max = 20000 / cnt
    if freq < allowed_freq_max and freq > allowed_freq_min and suitable_syns[0][1] > first_proximity:
        decision = True

    return decision


#Основная функия, принимаем на вход файл со статьёй, разбиваем на предложения и подбираем сленговые синонимы
def chiki_briki(filename):
    synonyms_cnt = 30
    
    target_article_txt = open(filename, 'r', encoding='utf8')
    target_article = target_article_txt.read()
    ta_sents = nltk.sent_tokenize(target_article)
    new_article = ''
    for sent in ta_sents:
        la = isp_api.lemmatizationAnnotate(text = sent)
        lp = isp_api.posTaggingAnnotate(text = sent)
        
        comparer = []
        new_sent = []       
        for i in range(len(la['annotations']['lemma'])):
            l = la['annotations']['lemma'][i]
            p = lp['annotations']['pos-token'][i]
            word_id = str(new_vocab.id_by_word(l['value'], tag = p['value']['tag']))
            comparer.append({'text': l['text'], 'norma': l['value'], 'tag': p['value']['tag'], 'word_id': word_id})
        
        for i, c in enumerate(comparer):
            if c['tag'] in ['PR', 'PART', 'CONJ'] or not c['text'].isalnum():
                new_sent.append(c['text'])
                continue
                
            word_id = new_vocab.id_by_word(c['norma'], tag = c['tag'])
            word_item = new_vocab.word_by_id(word_id, return_val = 'item')
            word_norm = new_vocab.word_by_id(word_id)
            if word_id is None or int(word_item['cnt']) < 10:
                new_sent.append(c['text'])
                continue
                
            #print(word_item['normal_form'] + ' - ' + word_item['origin'])
            ''' 
            if word_item['origin'] is not 'xax':
                new_sent.append(c['text'])
                continue
            '''
            #new_sent.append(c['norma'])
            synonyms = w2v_model.most_similar_cosmul(positive = [str(word_id)], topn = synonyms_cnt)
            '''
            print(word_item['normal_form'])
            for s in synonyms:
                print(new_vocab.word_by_id(int(s[0])), end = " - ")
                print(s[1])
            '''
            
            suitable_syns = [s for s in synonyms if new_vocab.word_by_id(int(s[0]), return_val = 'item')['origin'] in
                           ['txx'] and 
                           new_vocab.word_by_id(int(s[0]), return_val = 'item')['tags'] == c['tag'] and 
                           not (new_vocab.word_by_id(int(s[0])) == 'не' + word_norm or
                           'не' + new_vocab.word_by_id(int(s[0])) == word_norm or
                           new_vocab.word_by_id(int(s[0])) == 'без' + word_norm or
                           'без' + new_vocab.word_by_id(int(s[0])) == word_norm)]
            lemma = [w['word_id'] for w in comparer]
            '''
            ranked_syns = []
            for s in suitable_syns:
                lemma[i] = s[0]
                rank = (w2v_model.score([lemma])) / 100 
                ranked_syns.append({'syn': s[0], 'rank': rank[0]})
            
            ranked_syns = sorted(ranked_syns, key=lambda s: s['rank'], reverse=True)
            '''
            
            #print(c['norma'])
            #print(suitable_syns)
            #print(ranked_syns)
            
            if len(suitable_syns) == 0:  
                new_sent.append(c['text'])
                continue
            
            if  make_decision(suitable_syns, word_item, new_vocab.id_counter):
                synonym = suitable_syns[0][0]
                synonym_item = new_vocab.word_by_id(int(suitable_syns[0][0]), return_val = 'item')
                
                synonym_parse = get_propper_parse(synonym_item['normal_form'], synonym_item['word_forms'])
                word_parse = get_propper_parse(c['text'], word_item['word_forms'])
                synonym_parse = inflect_by_template(synonym_parse, word_parse)
                
                word_to_substitute = synonym_parse.word
                #Первое слово в предожении с заглавной буквы
                if i == 0:
                    word_to_substitute = word_to_substitute.title()
                    
                new_sent.append(word_to_substitute)
            #elif suitable_syns[0][1] > second_proximity: 
                #optionnal_syn = '(' + new_vocab.word_by_id(int(ranked_syns[0]['syn'])) + ')'
                #new_sent.append(optionnal_syn)
            else:  
                new_sent.append(c['text'])
                continue
                
        
        print(sent)
        new_sent = ' '.join(new_sent)
        print(new_sent)
        new_article += new_sent + ' '
        print('\n')
            
        '''
            t_id = new_vocab.id_by_word(t)
            
            
            if t_id is not None and int(new_vocab.word_by_id(t_id, return_val = 'item')['cnt']) > 10:
                lst = w2v_model.most_similar(positive = [str(t_id)])
                new_t = new_vocab.word_by_id(int(lst[0][0]), return_val = 'item')
                if new_t['origin'] == 'tax' or new_t['origin'] == 'txx':
                    print(new_t['normal_form'], end = ' ')
                else:
                    print(t, end = ' ')
        
            else:
                print(t, end = ' ')
        '''
    return new_article
    

new_article = chiki_briki('target_article2.txt')
new_article_txt = open('new_article.txt', 'w', encoding='utf8')
new_article_txt.write(new_article)

iPad Mini 4 стал логическим продолжением популярной линейки «яблочных» планшетов, которые нашли применение в самых разных сферах.
iPad Mini 4 стал логическим продолжением популярной линейки « яблочных » клав , которые глядывали применение в самых разных сферах .


Развлекательное устройство, «читалка», автомобильный навигатор, рабочий инструмент — компактный и лёгкий планшет стал настоящим хитом для тех, кто всюду носит с собой гаджет и нуждается в большем экране, чем у смартфона или смартпэда.
Развлекательное устройство , « читалка » , автомобильный навигатор , рабочий инструмент — компактный и лёгкий клава стал настоящим хитом для тех , кто всюду тягает с собой гаджет и нуждается в большем амольде , чем у смарта или смартпэда .


Между тем рынок чутко отреагировал на запросы пользователей и наполнился множеством самых разных аксессуаров, среди которых чехлы, которые позволяют ронять iPad Mini 4 без вреда для корпуса, закалённые стекла и целая плеяда полезных подставок на любой случай

2065

# НИЖЕ ЧЕРНОВИК, СМОТРЕТЬ ТУДА НЕ НУЖНО

In [15]:
lst = w2v_model.most_similar(positive = [str(new_vocab.id_by_word('samsung'))])
for l in lst:
    print(new_vocab.word_by_id(int(l[0])), end = " - ")
    print(l[1])

s8 - 0.7711077332496643
active - 0.7300979495048523
s6 - 0.7240570187568665
s3 - 0.7103427052497864
s9 - 0.7008739709854126
s7 - 0.6820350289344788
samsung - 0.677720844745636
galaxy - 0.6628284454345703
c - 0.6439862251281738
флагманский - 0.6419556736946106


In [141]:
print(new_vocab.id_by_word('samsung'))
print(new_vocab.id_by_word('galaxy'))
print(1 + w2v_model.score([[str(new_vocab.id_by_word('samsung')), str(new_vocab.id_by_word('mass'))]])[0] / 100)
print(w2v_model.most_similar_cosmul(positive = ['256']))

print(new_vocab.id_counter)
allowed_freq_min = 1000 / new_vocab.id_counter
allowed_freq_max = 10000 / new_vocab.id_counter
print(allowed_freq_min)
print(allowed_freq_max)

2034
2365
0.915373687744
[('2151', 0.8232387900352478), ('2367', 0.7768270969390869), ('2380', 0.7505201101303101), ('1583', 0.7477496862411499), ('1344', 0.7444450855255127), ('26640', 0.7400096654891968), ('453', 0.7385299801826477), ('2021', 0.7321900129318237), ('3844', 0.7259445190429688), ('7477', 0.724348783493042)]
87557
0.011421131377274232
0.11421131377274232


In [12]:
comments = ''
with open('learn_scrapy\pda.jl', 'rb') as f:
    for item in jl.reader(f):
        comments = comments + item['comment']       #создаём один string на все комменты
                                                    #скорее всего, будет логичнее для N-граммов каждое предложение
                                                    #запоминать в один string и хранить как list()

In [13]:
#тут токенизируем комментарии
#есть подозрение, что я создаю слишком много сущностей 

tknzr = nltk.TweetTokenizer()
print('Kek1')
tokens = tknzr.tokenize(comments)
print('Kek2')
text = nltk.Text(tokens)
print('Kek3')
sents = nltk.sent_tokenize(comments)                         #строить n-граммную модель, возможно, буду 
print('Kek4')                                                             #анализируя предложения по-отдельности
words = sorted([w.lower() for w in tokens if w.isalpha()])   #впоследствии буду отсекать наименее наиболее частотные слова

Kek1
Kek2
Kek3
Kek4


In [20]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

print(lexical_diversity(text))

0.10600020178435018


In [21]:
sent_tokens = list()                            #здесь храним предложения
for i in range(len(sents)):
    sent_tokens.append(tknzr.tokenize(sents[i]))    

In [22]:
#сох нормальные формы слов
morph = pm2.MorphAnalyzer()     
vocab_normalized = sorted(set([morph.parse(w)[0].normalized for w in words]), key = lambda w: w.normal_form)   

In [ ]:
def progress(full, curr, checkpoint):
    if curr / full * 100 > checkpoint:
        print(curr / full * 100, end = '')
        print('% complited')
        return True
    
    else:
        return False
    
i = 0
h = 0.1
checkpoint = h
time = tm.clock()
for sent in sents:
    if progress(len(sents), i, checkpoint):
        time_per_checkpoint = tm.clock() - time
        print('Time per checkpoint:', end = ' ')
        print(time_per_checkpoint)
        print('Time per sentence:', end = ' ')
        time_per_sent = time_per_checkpoint / (len(sents) * h)
        print(time_per_sent)
        print('Estimated time:', end = ' ')
        print(time_per_checkpoint * (100 - checkpoint))
        time = tm.clock()
        checkpoint = checkpoint + h
        
    toks = tknzr.tokenize(sent)
    toks = [t.lower() for t in toks if t.isalnum()]
    sentence = ''
    for t in toks:
        sentence = sentence + t + ' '
    
    lemma_raw = isp_api.lemmatizationAnnotate(text = sentence)
    lemmatized = ''
    if lemma_raw.get('annotations').get('lemma') is not None:
        for l in lemma_raw.get('annotations').get('lemma'):
            lemmatized = lemmatized + l.get('value') + ' '
        
        sents[i] = lemmatized
    
    else:
        sents[i] = ''
        
    i = i + 1
        
    
sents[0]

In [2]:
with open('lemmatized.txt', 'w', encoding='utf8') as f:
    for s in sents[0:5000]:
        f.write(s + '\n')

NameError: name 'sents' is not defined

In [11]:
model = gensim.models.Word2Vec(gensim.models.word2vec.LineSentence('lemmatized.txt'), min_count=10)


In [93]:
model.most_similar(positive = ['айфон'])

[('а', 0.9998720288276672),
 ('они', 0.9998717904090881),
 ('где', 0.9998696446418762),
 ('этот', 0.9998689293861389),
 ('мочь', 0.9998672604560852),
 ('когда', 0.999864935874939),
 ('это', 0.9998648166656494),
 ('там', 0.9998632669448853),
 ('с', 0.9998587965965271),
 ('ничто', 0.9998581409454346)]

In [5]:
i = 5

message = 'сейчас я говорю о том человеке который спит отдыхая на работе'
message_tokens = tokenizer.tokenize(message)
parse_tokens = [morph.parse(t) for t in message_tokens]
#print(parse_tokens[i])
isp_POS = isp_api.posTaggingAnnotate(text = message)
isp_lemma = isp_api.lemmatizationAnnotate(text = message)

#print(isp_POS['annotations']['pos-token'][i])

#print(isp_lemma.get('annotations').get('lemma')[i].get('value'))

#print(' '.join([l.get('value') for l in isp_lemma.get('annotations').get('lemma')]))

for j in range(len(message_tokens)):
    isp_lemma_token = isp_lemma.get('annotations').get('lemma')[j].get('value')
    morph_parse_token = [w.normalized for w in morph.parse(isp_lemma_token) if w.normal_form == w.word]
    print(isp_lemma_token)
    print(isp_POS['annotations']['pos-token'][j])
    print(morph_parse_token)
    print("\n")

сейчас
{'value': {'characters': [], 'tag': 'ADV', 'type': 'syn-tag-rus'}, 'start': 0, 'text': 'сейчас', 'end': 6, 'annotated-text': 'сейчас я говорю о том человеке который спит отдыхая на работе'}
[Parse(word='сейчас', tag=OpencorporaTag('ADVB'), normal_form='сейчас', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сейчас', 3, 0),))]


я
{'value': {'characters': [{'tag': 'Masculine', 'type': 'gender'}, {'tag': 'DEICTIC', 'type': 'pronoun'}, {'tag': 'Singular', 'type': 'number'}, {'tag': 'Nominative', 'type': 'case'}, {'tag': 'Animated', 'type': 'animacy'}], 'tag': 'S', 'type': 'syn-tag-rus'}, 'start': 7, 'text': 'я', 'end': 8, 'annotated-text': 'сейчас я говорю о том человеке который спит отдыхая на работе'}
[Parse(word='я', tag=OpencorporaTag('NPRO,1per sing,nomn'), normal_form='я', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'я', 3100, 0),))]


говорить
{'value': {'characters': [{'tag': 'First', 'type': 'person'}, {'tag': 'Singular', 'type': 'number'}, {'tag': 'NotPast', 'type

In [76]:
ws = ['встать', 'вставать', 'встали', 'вставали']
for w in ws:
    print(w)
    ns = morph.parse(w)
    ts = isp_api.lemmatizationAnnotate(text = w)
    print(ts)
    print(ns)
        
    print('\n')

встать
{'text': 'встать', 'annotations': {'lemma': [{'text': 'встать', 'value': 'вставать', 'annotated-text': 'встать', 'start': 0, 'end': 6}]}}
[Parse(word='встать', tag=OpencorporaTag('INFN,perf,intr'), normal_form='встать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'встать', 904, 0),))]


вставать
{'text': 'вставать', 'annotations': {'lemma': [{'text': 'вставать', 'value': 'вставать', 'annotated-text': 'вставать', 'start': 0, 'end': 8}]}}
[Parse(word='вставать', tag=OpencorporaTag('INFN,impf,intr'), normal_form='вставать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'вставать', 903, 0),))]


встали
{'text': 'встали', 'annotations': {'lemma': [{'text': 'встали', 'value': 'вставать', 'annotated-text': 'встали', 'start': 0, 'end': 6}]}}
[Parse(word='встали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='встать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'встали', 904, 4),))]


вставали
{'text': 'вставали', 'annotations': {'lemma': [{'text': 'встав